### Load tensorflow

In [32]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [20]:
data = pd.read_csv('prices.csv')
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [21]:
data.shape

(851264, 7)

In [22]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [23]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [24]:
data.drop(['date','symbol'], axis=1, inplace = True)

In [25]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [26]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [27]:
data_1000 = data.head(1000)
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(data_1000)
df_normalized = pd.DataFrame(np_scaled, index=data_1000.index, columns=data_1000.columns)
df_normalized.head()
print(df_normalized.head())
print(df_normalized.shape)
df_normalized = df_normalized.astype(np.float32)

       open     close       low      high    volume
0  0.194837  0.198723  0.193984  0.198503  0.009988
1  0.197730  0.189349  0.190178  0.197372  0.011022
2  0.183569  0.181303  0.182133  0.188135  0.011500
3  0.182130  0.183974  0.179837  0.184472  0.009259
4  0.184576  0.181335  0.180784  0.184297  0.006487
(1000, 5)


### Divide the data into train and test sets

In [28]:
X=df_normalized.drop(["close"],axis=1)
y=df_normalized["close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(700, 4) (700,)
(300, 4) (300,)


## Building the graph in tensorflow

1.Define input data placeholders

In [13]:
n_features = X.shape[1]
tf.reset_default_graph()

In [14]:
X_in = tf.placeholder(tf.float32, [None, n_features], "X_in") 
y_in = tf.placeholder(tf.float32, [None], "y_in")

2.Define Weights and Bias

In [15]:
w = tf.Variable(tf.random_normal([n_features, 1]), name="w")
b = tf.Variable(np.random.randn(), name="b")

Instructions for updating:
Colocations handled automatically by placer.


3.Prediction

In [16]:
h = tf.add(tf.matmul(X_in, w), b)

4.Loss (Cost) Function [Mean square error]

In [17]:
cost = tf.reduce_mean(tf.square(tf.subtract(y_in, h)))

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [18]:
optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(cost)

## Execute the Graph for 100 epochs and observe the loss

In [19]:
init = tf.global_variables_initializer()
training_epochs = 100
# Launch the graph 
with tf.Session() as sess: 
	# Load initialized variables in current session 
	sess.run(init) 

	# Fit all training data 
	for epoch in range(training_epochs): 

		# perform gradient descent step 
		sess.run(optimizer, feed_dict={X_in: X_train, y_in: y_train}) 
		
		# Display logs per epoch step 
		if (epoch+1) % 10 == 0: 
			c = sess.run(cost, feed_dict={X_in: X_train, y_in: y_train}) 
			print("Epoch:{0:6} \t Cost:{1:10.4} \t W:{1:6.4} \t b:{3:6.4}". 
				format(epoch+1, c, sess.run(w), sess.run(b))) 
			
	# Print final parameter values 
	print("Optimization Finished!") 
	training_cost = sess.run(cost, feed_dict={X_in: X_train, y_in: y_train}) 
	print("Final training cost:", training_cost, "w:", sess.run(w), "b:", 
		sess.run(b), '\n') 
	
	# Testing the model 
	testing_cost = sess.run(tf.reduce_mean(tf.square(tf.subtract(y_in, h))), 
							feed_dict={X_in: X_test, y_in: y_test}) 
	
	print("Final testing cost:", testing_cost) 
	


Epoch:    10 	 Cost:   0.07005 	 W:0.07005 	 b:0.4279
Epoch:    20 	 Cost:   0.05583 	 W:0.05583 	 b:0.3899
Epoch:    30 	 Cost:   0.04507 	 W:0.04507 	 b:0.3568
Epoch:    40 	 Cost:   0.03693 	 W:0.03693 	 b:0.3279
Epoch:    50 	 Cost:   0.03076 	 W:0.03076 	 b:0.3028
Epoch:    60 	 Cost:   0.02607 	 W:0.02607 	 b:0.2808
Epoch:    70 	 Cost:    0.0225 	 W:0.0225 	 b:0.2617
Epoch:    80 	 Cost:   0.01977 	 W:0.01977 	 b:0.2451
Epoch:    90 	 Cost:   0.01768 	 W:0.01768 	 b:0.2305
Epoch:   100 	 Cost:   0.01606 	 W:0.01606 	 b:0.2178
Optimization Finished!
Final training cost: 0.016063176 w: [[ 0.04623254]
 [-0.24945664]
 [-0.6426267 ]
 [-0.9064952 ]] b: 0.21777837 

Final testing cost: 0.020914663


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [49]:
sess = tf.Session()
sess.run(init)
print(sess.run(w))
print(sess.run(b))
sess.close()

[[ 0.67900807]
 [-0.26074335]
 [-1.3494318 ]
 [ 0.9238619 ]]
-0.72633976


### Find the Absolute mean square loss difference between training and testing loss.

In [50]:
print("Absolute mean square loss difference:", abs(training_cost - testing_cost)) 

Absolute mean square loss difference: 0.0043261433


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [15]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.optimizers import SGD 
from keras import utils 

Using TensorFlow backend.


### Execute the model

In [16]:
graph = tf.get_default_graph()

with graph.as_default():
        # build a sequential model 
        model = Sequential() # the first layer has to specify the dimensions of the input vector 
        model.add( Dense( units = 1, activation ='sigmoid', input_shape =(4,))) # add dropout layer for preventing overfitting 
        model.add( Dense( units = 1, activation ='softmax')) # print the summary of our model 
        model.summary() 

        # compile the model 
        model.compile( loss ='mse', optimizer = SGD(), metrics =['accuracy']) 

        # train the model 
        model.fit(X_train, y_train,epochs = 10, batch_size = 100) 

        # evaluate the model and print the accuracy score 

        scores = model.evaluate(X_test, y_test)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
700/700 [==============================] - 1s 714us/step - loss: 0.8155 - acc: 0.0014
Epoch 2/10
700/700 [==============================] - 0s 29us/step - loss: 0.8155 - acc: 0.0014
Epoch 3/10
700/700 [==============================] - 0s 29us/step - loss: 0.8155 - acc: 0.0014
Epoch 4/10
700/700 [==============================] - 0s 29us/step - loss: 0.8155 - acc: 0.0014
Epoch 5/10
700/700 [===================

### Classification using Keras 

In [46]:
print('loss:', scores[ 0]) 
print('accuracy:', scores[ 1])
y_pred=model.predict(X_test)
rms=np.sqrt(np.mean(np.power((np.array(y_test)-np.array(y_pred)),2)))
print("rmse:", rms[0])

loss: 0.800693789323171
accuracy: 0.0
rmse: [0.899809]


### Load the given Iris data using pandas (Iris.csv)

In [23]:
dfIris = pd.read_csv("Iris.csv")
dfIris.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [24]:
X = dfIris.drop(["Id","Species"],axis=1)
y = dfIris["Species"]

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [35]:
y = pd.get_dummies(dfIris['Species'])

### Divide the dataset into Training and test (70:30)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(105, 4) (105, 3)
(45, 4) (45, 3)


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

### Fitting the model and predicting 

In [38]:
# define some hyper parameters 
batch_size = 100 
n_inputs =  4
n_classes = 3 
n_epochs = 10

graph = tf.get_default_graph()

with graph.as_default():
     # build a sequential model 
            model = Sequential() # the first layer has to specify the dimensions of the input vector 
            model.add( Dense( units = 10, activation ='sigmoid', input_shape =(n_inputs,))) # add dropout layer for preventing overfitting 
            model.add( Dense( units = 8, activation ='sigmoid')) 
            model.add( Dense( units = n_classes, activation ='softmax')) # print the summary of our model 
            model.summary() 

            # compile the model 
            model.compile( loss ='categorical_crossentropy', optimizer = SGD(), metrics =['accuracy']) 


            # train the model 
            model.fit(X_train, y_train, batch_size = batch_size, epochs = n_epochs) 

            # evaluate the model and print the accuracy score 

            scores = model.evaluate(X_test, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 27        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
105/105 [==============================] - 1s 6ms/step - loss: 1.1569 - acc: 0.3143
Epoch 2/10
105/105 [==============================] - 0s 57us/step - loss: 1.1549 - acc: 0.3143
Epoch 3/10
105/105 [==============================] - 0s 57us/step - loss: 1.1529 - acc: 0.3143
Epoch 4/10
105/105 [==============================] - 0s 57us/step - loss: 1.1518 - acc: 0.3143
Epoch 5/10
105/105 [=====

### Report Accuracy of the predicted values

In [39]:
print('loss:', scores[ 0]) 
print('accuracy:', scores[ 1])

loss: 1.1166844818327162
accuracy: 0.37777778572506376
